In [11]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# Define the true function to optimize
def f(x):
    return np.exp(-(x-1.654)**2/.1) + 0.3 * np.sin(5*x)

# Initial training data
X_train_initial = np.array([[0], [3]])
y_train_initial = f(X_train_initial).ravel()

# Define the range for prediction and acquisition function
X = np.linspace(0, 5, 400).reshape(-1, 1)
y_true = f(X).ravel()

# Define the Gaussian Process model
kernel = Matern(nu=2.5)

# Number of iterations to run
num_iterations = 16

# Storage for data at each iteration
iterations_data = []
X_train = X_train_initial.copy()
y_train = y_train_initial.copy()
max_values = [np.max(y_train[:i+1]) for i in range(len(y_train))]

# Run the optimization process and store data for each iteration
for i in range(num_iterations):
    # Fit the GP model
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
    gp.fit(X_train, y_train)
    
    # Predict mean and standard deviation
    mu, sigma = gp.predict(X, return_std=True)
    
    # Calculate the Expected Improvement (EI)
    xi = 0.01  # Exploration-exploitation trade-off parameter
    y_max = np.max(y_train)
    Z = (mu - y_max - xi) / sigma
    ei = (mu - y_max - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma == 0.0] = 0.0
    
    # Store current state
    iterations_data.append({
        'X_train': X_train.copy(),
        'y_train': y_train.copy(),
        'mu': mu.copy(),
        'sigma': sigma.copy(),
        'ei': ei.copy(),
        'y_max': y_max,
        'max_values': max_values.copy()
    })
    
    # Find the next suggested value
    next_x = X[np.argmax(ei)]
    
    # Update training data
    X_train = np.vstack((X_train, next_x))
    y_train = np.append(y_train, f(next_x))
    
    # Update max values
    max_values.append(np.max(y_train))

# Create figure with subplots


# Function to update the figure based on slider
def update_figure(iteration):
    data = iterations_data[iteration]
    X_train = data['X_train']
    y_train = data['y_train']
    mu = data['mu']
    sigma = data['sigma']
    ei = data['ei']
    y_max = data['y_max']
    max_vals = data['max_values']
    
    fig = make_subplots(rows=3, cols=1, 
                    subplot_titles=("Gaussian Process Model", 
                                   "Expected Improvement", 
                                   "Maximum Value Evolution"),
                    vertical_spacing=0.1,
                    row_heights=[0.5, 0.25, 0.25])

    # Initialize with empty traces that will be updated by the slider
    # Main plot - GP model
    fig.add_trace(go.Scatter(x=X.ravel(), y=y_true, mode='lines', line=dict(color='red', dash='dot'), 
                            name='True Function'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='blue'), name='GP Mean'), 
                row=1, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='blue', width=0), 
                            showlegend=False, name='Upper Bound'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', fill='tonexty', 
                            fillcolor='rgba(0, 0, 255, 0.2)', line=dict(color='blue', width=0),
                            showlegend=False, name='Lower Bound'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='markers', marker=dict(color='red', size=10),
                            name='Training Data'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='black', dash='dash'),
                            name='Max Observed Value'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='gray', dash='dash'),
                            name='Next Point'), row=1, col=1)

    # EI plot
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='green'),
                            name='Expected Improvement'), row=2, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='gray', dash='dash'),
                            showlegend=False), row=2, col=1)

    # Max value plot
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', 
                            line=dict(color='magenta'), marker=dict(color='magenta', size=10),
                            name='Max Value Evolution'), row=3, col=1)
    
    # If not the last iteration, get the next point
    next_point = None
    if iteration < len(iterations_data) - 1:
        next_training_points = iterations_data[iteration + 1]['X_train']
        next_point = next_training_points[-1]
    
    # Update main plot
    with fig.batch_update():
        # GP mean
        fig.data[1].x = X.ravel()
        fig.data[1].y = mu
        
        # Confidence intervals
        fig.data[2].x = X.ravel()
        fig.data[2].y = mu + 1.96 * sigma
        fig.data[3].x = X.ravel()
        fig.data[3].y = mu - 1.96 * sigma
        
        # Training data
        fig.data[4].x = X_train.ravel()
        fig.data[4].y = y_train
        
        # Max observed value
        fig.data[5].x = [X.min(), X.max()]
        fig.data[5].y = [y_max, y_max]
        
        # Next point vertical line
        if next_point is not None:
            fig.data[6].x = [next_point[0], next_point[0]]
            fig.data[6].y = [min(mu - 1.96 * sigma).min(), max(y_true.max(), (mu + 1.96 * sigma).max())]
        else:
            fig.data[6].x = []
            fig.data[6].y = []
        
        # EI plot
        fig.data[7].x = X.ravel()
        fig.data[7].y = ei
        
        if next_point is not None:
            fig.data[8].x = [next_point[0], next_point[0]]
            fig.data[8].y = [0, ei.max()]
        else:
            fig.data[8].x = []
            fig.data[8].y = []
        
        # Max value evolution
        iterations = np.arange(1,len(max_vals)+2)
        fig.data[9].x = iterations
        fig.data[9].y = max_vals
        
        fig.update_layout(
            height=800,
            width=800,
            title="Bayesian Optimization with Expected Improvement",
        )

        # Set y-axis ranges
        fig.update_yaxes(title_text="f(X)", row=1, col=1)
        fig.update_yaxes(title_text="Expected Improvement", row=2, col=1)
        fig.update_yaxes(title_text="Max Observed Value", row=3, col=1)

        # Set x-axis ranges and titles
        fig.update_xaxes(range=[0, 5.1], row=1, col=1)
        fig.update_xaxes(range=[0, 5.1], title_text="X", row=2, col=1)
        fig.update_xaxes(title_text="Number of Samples", row=3, col=1)
        return fig

import plotly.io as pio
figs = [update_figure(i) for i in range(num_iterations)]
for i, fig in enumerate(figs):
    pio.write_html(fig, f'figure_{i}.html')